# PyTorch 模型构建与理解指南

> 目标：帮助具备基础编程背景的工程师从零梳理“建模 → 训练 → 分析 → 选择 → 工程化”全流程，理解各环节的 PyTorch API 组合方式，避免盲目堆砌代码。


## 快速导航

- [1. 构建整体认知](#1-构建整体认知)
- [2. 数据与问题建模](#2-数据与问题建模)
- [3. 神经网络基本骨架](#3-神经网络基本骨架)
- [4. 训练与验证流程](#4-训练与验证流程)
- [5. 模型分析与调试](#5-模型分析与调试)
- [6. 模型选择策略](#6-模型选择策略)
- [7. PyTorch 生态与组件矩阵](#7-pytorch-生态与组件矩阵)
- [8. 工程化与持续改进](#8-工程化与持续改进)
- [9. 典型任务端到端范式](#9-典型任务端到端范式)


## 1. 构建整体认知

1. **问题 → 任务类型**：明确是监督/无监督/强化/生成任务；进一步细分为分类、回归、检测、分割、序列建模等。
2. **建模假设**：选择贴合任务的数据表示、损失函数及评价指标。KISS 原则下，先尝试最简单的可行方案。
3. **资源与约束**：估算可用 GPU/CPU、存储、推理时延、数据隐私约束等，影响模型大小与训练策略。
4. **工具选择**：基于任务特征与团队熟悉度，在原生 PyTorch 与高层框架（Lightning、TorchX）之间权衡，遵循 YAGNI，先用 PyTorch 原生 API 打通流程。
### 1.1 典型建模流水线
1. **准备数据**：收集 → 清洗 → 划分 → 封装 `Dataset/DataLoader`；必要时构建 DataPipe 以流式读取海量数据。
2. **定义模型**：基于 `nn.Module` 搭建骨架；优先从简，确保前向计算正确，使用单元测试验证核心算子。
3. **选择损失与指标**：将业务目标映射为可优化的损失，例如分类任务用 `nn.CrossEntropyLoss`，回归用 `nn.MSELoss`；同时定义离线指标（ACC、F1、BLEU 等）。
4. **配置优化器与调度器**：选择 `torch.optim` 中的优化算法、学习率调度策略，记录默认超参方便后续对比。
5. **训练循环**：编写 `train`/`evaluate` 函数；必要时加入混合精度、梯度裁剪、分布式逻辑。
6. **分析调试**：监控损失、指标、梯度；排查数值问题、数据质量；通过可视化辅助定位瓶颈。
7. **迭代与部署**：根据分析结果调整模型或数据；当离线指标满足要求，导出 ONNX/TorchScript，接入线上；持续监控回流数据。

### 1.2 迭代节奏示例
- 第 0 周：跑通最小基线（小模型 + 小数据批次），验证工程链路。
- 第 1~2 周：扩充数据增强、尝试更强 Backbone，同时保留指标对比表，确保改动有依据。
- 第 3 周起：引入自动化评估、A/B 测试、模型卡文档，形成闭环。


## 2. 数据与问题建模

### 2.1 数据理解
- **探索**：使用 `pandas`, `matplotlib`, `torch.utils.data` 等分析样本分布、标签平衡、缺失值。
- **特征工程**：图像/音频侧关注采样率、尺寸、标准化；表格数据关注归一化、离散化；文本需要分词、子词编码。
- **划分策略**：`train/val/test` 严格独立，时间序列需遵守时间顺序，交叉验证可用于小数据集评估稳定性。

### 2.2 数据加载
```python
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, data, transforms=None):
        self.data = data
        self.transforms = transforms
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        x, y = self.data[idx]
        if self.transforms:
            x = self.transforms(x)
        return x, y

train_loader = DataLoader(dataset=train_set, batch_size=64, shuffle=True,
                          num_workers=4, pin_memory=True)
```
- 结合 `torchvision.transforms`, `torchaudio.transforms`, `torchtext.data` 等垂类库提升数据预处理效率。
- 对于多模态或大规模数据，可引入 `IterableDataset`、`TorchData DataPipe`，保持流式处理。


## 3. 神经网络基本骨架

### 3.1 模块化设计
- 遵循 **单一职责**：将特征提取、分类头、损失函数、评估逻辑划分到独立组件。
- 通过 `nn.Module` 组合形成主模型，便于后续替换或扩展。

### 3.2 最小化模型示例
```python
import torch
from torch import nn
from torch.nn import functional as F

class Backbone(nn.Module):
    def __init__(self, in_channels=1, hidden=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        self.out_dim = 64
    def forward(self, x):
        return self.net(x).flatten(1)

class Classifier(nn.Module):
    def __init__(self, backbone, num_classes):
        super().__init__()
        self.backbone = backbone
        self.head = nn.Linear(backbone.out_dim, num_classes)
    def forward(self, x):
        features = self.backbone(x)
        return self.head(features)

model = Classifier(Backbone(in_channels=1), num_classes=10)
```
- `forward` 中避免副作用，使用 `__init__` 定义子模块。
- 若需要可扩展性，可引入策略模式（如不同 `Backbone` 子类），符合 SOLID 的开闭原则。


## 4. 训练与验证流程

### 4.1 基础训练循环
```python
def train_one_epoch(model, loader, criterion, optimizer, scaler=None):
    model.train()
    total_loss = 0.0
    for inputs, targets in loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad(set_to_none=True)
        with torch.autocast(device_type=device.type, enabled=scaler is not None):
            outputs = model(inputs)
            loss = criterion(outputs, targets)
        if scaler:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()
        total_loss += loss.item() * inputs.size(0)
    return total_loss / len(loader.dataset)
```
- 混合精度：`torch.cuda.amp.GradScaler`/`autocast` 减少显存占用。
- 梯度裁剪：`torch.nn.utils.clip_grad_norm_` 防止梯度爆炸。

### 4.2 验证与指标
```python
@torch.no_grad()
def evaluate(model, loader, metrics):
    model.eval()
    for metric in metrics:
        metric.reset()
    for inputs, targets in loader:
        preds = model(inputs.to(device))
        for metric in metrics:
            metric.update(preds, targets.to(device))
    return {metric.name: metric.compute() for metric in metrics}
```
- 指标库可自定义，也可使用 `torchmetrics` 或 `TorchEval`。
- 训练过程建议记录：Loss、主要指标、学习率、显存占用（`torch.cuda.memory_stats`）。

### 4.3 组织训练脚本
- 配置管理：用 `yaml` 或 `argparse` 分离超参，避免硬编码。
- 日志：`TensorBoard`, `wandb`, `mlflow` 等，至少使用 `SummaryWriter` 记录曲线。
- 检查点：`torch.save({'model': model.state_dict(), ...}, path)`，并设计恢复逻辑。


## 5. 模型分析与调试

1. **训练动态**：观察 Loss 曲线、梯度范数、学习率，定位欠拟合/过拟合。
2. **样本级分析**：
   - 误分类样本：保存错误样本，结合 `matplotlib`/`seaborn` 可视化。
   - 特征可视化：`torchvision.utils.make_grid`，`tsne`/`umap` 分析隐藏层表示。
3. **梯度与权重检查**：
   - `torch.autograd.gradcheck`/`gradgradcheck` 验证自定义算子。
   - 统计权重、梯度分布（直方图）检测梯度消失/爆炸。
4. **性能剖析**：
   - `torch.profiler.profile` + TensorBoard 追踪算子耗时。
   - `torch.utils.benchmark` 评估不同实现方案。
5. **数值稳定性**：关注 `torch.isfinite`, `torch.nan_to_num`，必要时增加正则或梯度裁剪。
6. **部署一致性**：使用 `torch.onnx`、`torch.export` 或 `torch.jit` 验证前向图，确保推理结果与训练端一致。


## 6. 模型选择策略

| 场景 | 基线模型 | 进阶方向 | 关键考虑 |
| --- | --- | --- | --- |
| 图像分类 | `ResNet`, `EfficientNet` | 视觉 Transformer、蒸馏 | 数据增广、推理时延、分辨率 |
| 目标检测 | `Faster R-CNN`, `YOLOv5` | `DETR`, `Mask R-CNN` | 标注粒度、NMS 阈值、吞吐 |
| 语音识别 | CTC + `DeepSpeech` | `Conformer`, `RNN-T` | 声学特征、流式需求 |
| NLP 分类 | `BiLSTM`, `TextCNN` | `BERT`/`RoBERTa` 微调 | 预训练资源、序列长度 |
| 推荐系统 | DLRM, `DeepFM` | `TorchRec` 分布式嵌入 | 特征规模、冷启动 |
| 强化学习 | DQN, PPO | 分布式 RL (`TorchRL`) | 环境复杂度、样本效率 |

- **筛选标准**：
  1. **指标目标**：精度、召回、F1、BLEU、ROC-AUC 等。
  2. **资源成本**：显存、训练时长、推理延迟、可移植性。
  3. **可解释性**：业务监管要求可解释时偏向简单模型或引入可解释方法。
  4. **数据规模**：样本量低时先用轻量模型并结合交叉验证。
- **迭代策略**：
  - 先跑开源预训练模型做对比，确保自研模型有明确优势。
  - 保留多组实验配置并记录指标，方便回溯（可用 `hydra` + `mlflow`）。


## 7. PyTorch 生态与组件矩阵

| 需求 | 推荐组件 | 核心能力 | 备注 |
| --- | --- | --- | --- |
| 视觉 | `torchvision` | 数据集、模型、增广 | `transforms`, `models`, `ops` |
| 音频 | `torchaudio` | I/O、谱图、预训练模型 | 结合 `Kaldi` 风格接口 |
| 自然语言 | `torchtext` | 词表、数据 Iterator | 亦可结合 Hugging Face `datasets` |
| 推荐系统 | `TorchRec` | 大规模稀疏嵌入、分布式 | 与 `torch.distributed`、FSDP 配套 |
| 数据流水线 | `TorchData` | DataPipe、可组合 ETL | 适合大文件与流式数据 |
| 强化学习 | `TorchRL` | 环境封装、策略模板 | 支持分布式、伪代码到实现 |
| 微调/压缩 | `TorchTune`, `torch.amp`, `torch.quantization` | LoRA/QLoRA、量化、蒸馏 | 与 `torch.compile` 协同 |
| 编译优化 | `torch.compile`, `torch.fx`, `torch._dynamo` | 图级优化、裁剪 | 在推理部署前评估吞吐 |

- 组合策略：以主干 PyTorch API 为核心，按需引用专用库；避免一次性引入所有组件造成复杂度激增。
- 参考 `pytorchLibrary/BasciApi/ApiInstruction.ipynb` 获取 API 索引，按章节回溯具体函数用法。


## 8. 工程化与持续改进

1. **可重复性**：
   - 固定随机种子：`torch.manual_seed`, `torch.cuda.manual_seed_all`。
   - 设置确定性：`torch.use_deterministic_algorithms(True)`，关注性能权衡。
2. **配置与版本管理**：
   - 使用 `requirements.txt`/`conda env` 锁定依赖。
   - 通过 `git` 维护实验分支，遵循单一职责提交。
3. **测试与验证**：
   - 单元测试：`pytest` + `torch.testing.assert_close` 验证模块输出。
   - 集成测试：构造最小数据集跑通训练1-2个 iteration，检测接口变化。
4. **部署路径**：
   - `torch.jit.trace`/`script` → C++/LibTorch。
   - `torch.onnx.export` → 推理框架（ONNX Runtime / TensorRT）。
   - `torch.export` (PyTorch 2) → 统一导出 IR。
5. **性能监控**：
   - 在线推理需记录吞吐、延迟、错误率（Prometheus + Grafana）。
   - 结合 `torch.monitor`/`torch.profiler` 定期定位瓶颈。
6. **知识沉淀**：
   - 维护模型卡（Model Card），记录用途、数据、评估、限制。
   - 复盘失败实验，完善决策依据。

> 建议从一个最小工作示例出发（30 分钟内可跑完一个 epoch），每次迭代仅改变一个因素，持续验证假设，避免违背 DRY/KISS 原则。


## 9. 典型任务端到端范式

> 以下示例从“数据 → 模型 → 训练 → 分析 → 扩展”五个环节拆解典型深度学习任务，帮助理解如何把上文原则落地。每个案例均遵循相同套路：先建立可运行的最小版本，再逐步替换组件提升效果。


### 9.1 图像分类（CNN + torchvision）

**目标**：手写数字分类（MNIST）→ 迁移到真实业务（自定义图像）

| 环节 | 关键库/方法 | 说明 |
| --- | --- | --- |
| 数据 | `torchvision.datasets.MNIST`, `transforms.Compose` | 自动下载并应用增广/标准化 |
| 模型 | 自定义 `nn.Module` 或 `torchvision.models.resnet18` | 从简单 CNN 起步，再尝试迁移学习 |
| 损失/指标 | `nn.CrossEntropyLoss`, Accuracy | 分类任务的基础配置 |
| 训练 | `torch.optim.Adam`, `StepLR`, AMP | 典型优化组合；对大模型使用混合精度 |
| 分析 | `TensorBoard`, 错误样本可视化 | 观察学习曲线、误分类图像 |

**手把手流程**
1. **准备数据**：
```python
from torchvision import datasets, transforms

train_tfms = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
train_ds = datasets.MNIST(root='data', train=True, download=True, transform=train_tfms)
val_ds = datasets.MNIST(root='data', train=False, download=True,
                        transform=transforms.ToTensor())
```
2. **封装 DataLoader**：`DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=4)`。
3. **搭建模型骨架**：先使用上文 `Backbone` + `Classifier`，确认能收敛；再替换为 `torchvision.models.resnet18(weights='IMAGENET1K_V1')` 并冻结/解冻层实现迁移学习。
4. **训练循环**：复用 `train_one_epoch`/`evaluate`，加入学习率调度：
```python
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
```
5. **分析调试**：
   - 画 loss/acc 曲线：`SummaryWriter.add_scalar`。
   - 导出误分类图像：对 `pred != label` 的样本使用 `torchvision.utils.make_grid` 保存。
   - 性能瓶颈：若训练慢，开启 `torch.backends.cudnn.benchmark = True` 或裁剪输入尺寸。
6. **扩展**：
   - 引入 `AutoAugment`, `RandAugment`；
   - 组合 `torch.compile` 提升推理吞吐；
   - 导出 `torch.onnx.export`，部署到 TensorRT。


### 9.2 文本分类（TorchText + 预训练模型）

**目标**：新闻主题分类（AG_NEWS）→ 迁移到中文情感分析

| 环节 | 关键库/方法 | 说明 |
| --- | --- | --- |
| 数据 | `torchtext.datasets.AG_NEWS`, `build_vocab_from_iterator` | 英文示例；中文可用 `THUCNews`/自建语料 |
| 模型 | `nn.EmbeddingBag + Linear` 基线；`transformers.BertForSequenceClassification` 进阶 | 先对比轻量/重型模型 |
| 损失/指标 | `nn.CrossEntropyLoss`, Accuracy/F1 | 文本类别不均时关注 F1 |
| 训练 | `torch.optim.SGD`/`AdamW`, `transformers.Trainer` | 小模型手写循环，大模型可用 HF Trainer |
| 分析 | 混淆矩阵、关键词权重（attention 可视化） | 帮助定位误判类别 |

**基线实现**
```python
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

train_iter = AG_NEWS(split='train')
tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])
```
1. **向量化**：将文本转索引序列，使用 `EmbeddingBag` 实现平均池化：
```python
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)
```
2. **训练**：构造批次时保留 `offsets`，调用通用 `train_one_epoch`。
3. **进阶**：引入 `transformers`：
```python
from transformers import BertTokenizerFast, BertForSequenceClassification, TrainingArguments, Trainer
```
   - 准备数据集 → `DatasetDict` → Tokenizer → `Trainer`。
   - 调整学习率 (`2e-5` 起步)、warmup 步数、梯度累积，避免显存不足。
4. **分析调试**：计算分类报告、绘制混淆矩阵；通过 attention 可视化了解模型关注词；监控 `loss` 与 `learning_rate`。
5. **扩展**：蒸馏 (`DistilBERT`)、量化（`torch.quantization.quantize_dynamic`）、多语言模型（mBERT）。


### 9.3 语音识别（Torchaudio + 预训练声学模型）

**目标**：将语音波形转文字，先使用公开集（LibriSpeech）+ CTC 基线，再升级到 wav2vec2 预训练模型。

| 环节 | 关键库/方法 | 说明 |
| --- | --- | --- |
| 数据 | `torchaudio.datasets.LIBRISPEECH` | 提供音频 + 文本标签 |
| 特征 | `torchaudio.transforms.MelSpectrogram` | 将波形转梅尔频谱 |
| 模型 | `nn.Sequential(CNN + BiLSTM + Linear)` 基线；`torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H` | 预训练模型显著提升准确率 |
| 损失 | `nn.CTCLoss` | 处理长度不对齐问题 |
| 训练 | `torch.optim.AdamW`, Grad Accumulation | 音频长 → 需梯度累积 |
| 分析 | 词错误率 (WER), 字错误率 (CER) | 语音识别关键指标 |

**基线步骤**
1. **加载数据**：
```python
import torchaudio
train_ds = torchaudio.datasets.LIBRISPEECH('data', url='train-clean-100', download=True)
```
2. **预处理**：
```python
mel = torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=80)
```
   - 对音频执行 `mel(waveform)`；
   - 文本转字母索引，可自建 `TextTransform`。
3. **模型骨架**：CNN 压缩时间维 → BiLSTM 捕捉上下文 → 全连接输出字母概率，前向输出 `(T, N, C)`。
4. **训练循环**：
```python
criterion = nn.CTCLoss(blank=0)
loss = criterion(log_probs, targets, input_lengths, target_lengths)
```
   - 由于句子长度不同，需要维护 `input_lengths` 与 `target_lengths`。
5. **进阶（wav2vec2 微调）**：
```python
bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
model = bundle.get_model().to(device)
processor = bundle.get_text_processor()
```
   - 冻结部分层、降低学习率 (`1e-5`)，使用 `AdamW`。
6. **分析调试**：评估 WER/CER；对错误案例回听音频，辨析是发音还是文本标注问题；使用 `torch.profiler` 检查算子耗时。
7. **部署**：将模型导出 TorchScript (`torch.jit.script`)，结合 `torchaudio.io.StreamReader` 构建实时推理。


### 9.4 强化学习（TorchRL + DQN）

**目标**：在 OpenAI Gym `CartPole-v1` 环境中训练智能体保持平衡。

| 环节 | 关键库/方法 | 说明 |
| --- | --- | --- |
| 环境 | `gymnasium.make`, `torchrl.envs` | 包装环境以兼容批处理 |
| 模型 | `torch.nn.Sequential` (MLP) | 状态 → 动作 logits |
| 算法 | `torchrl.objectives.DQNLoss` | 实现 Q-learning |
| 数据收集 | `torchrl.collectors.SyncDataCollector` | 自动执行环境交互 |
| 策略改进 | ε-greedy, 目标网络 | 稳定训练 |

**步骤概览**
1. **环境与策略**：
```python
from torchrl.envs import EnvCreator, ParallelEnv
import gymnasium as gym

env = ParallelEnv(4, EnvCreator(lambda: gym.make('CartPole-v1')))
```
2. **网络定义**：
```python
policy_net = nn.Sequential(
    nn.Linear(4, 128), nn.ReLU(),
    nn.Linear(128, 2)
)
```
3. **损失与优化器**：
```python
from torchrl.objectives import DQNLoss
loss_module = DQNLoss(policy_net, action_space=2)
optimizer = torch.optim.Adam(policy_net.parameters(), lr=1e-3)
```
4. **数据采集与训练循环**：
```python
from torchrl.collectors import SyncDataCollector
collector = SyncDataCollector(
    env,
    policy_net,
    frames_per_batch=1000,
    total_frames=100_000,
    exploration_type='epsilon_greedy',
    epsilon_init=0.9,
    epsilon_end=0.05,
    epsilon_decay=50_000
)
```
   - 对每个批次：计算损失、反向传播、软更新目标网络。
5. **分析调试**：跟踪平均回合（episode）奖励；使用 `wandb` 绘制学习曲线；检查价值函数是否发散（Q 值过大）。
6. **扩展**：切换为 PPO/Advantage Actor-Critic；多环境并行 (`AsyncCollector`)，或使用 `TorchRL` 提供的库内策略模板。


### 9.5 推荐系统（TorchRec + DLRM）

**目标**：构建点击率预估模型，适用于大规模稀疏特征。

| 环节 | 关键库/方法 | 说明 |
| --- | --- | --- |
| 数据 | Criteo 1TB / 自建日志；`TorchRec.datasets` | 特征包含数百个稀疏类别 |
| 模型 | `torchrec.models.dlrm.DLRM` | 工业级基准模型 |
| 特征表示 | `EmbeddingBagCollection`, `KeyedJaggedTensor` | 统一管理多字段嵌入 |
| 训练 | `torchrec.distributed.FSDP`, `OSS` | 解决大规模参数并行 |
| 分析 | AUC, calibration, offline/online A/B | 推荐系统特有指标 |

**实施步骤**
1. **特征工程**：
   - 将稀疏特征映射到 `KeyedJaggedTensor`；连续特征标准化。
2. **嵌入配置**：
```python
from torchrec.modules.embedding_modules import EmbeddingBagCollection, EmbeddingBagConfig

embedding_bag_collection = EmbeddingBagCollection(
    tables=[
        EmbeddingBagConfig(
            name="user_table",
            feature_names=["user_id"],
            num_embeddings=users,
            embedding_dim=64,
            pooling="sum"
        ),
        EmbeddingBagConfig(
            name="item_table",
            feature_names=["item_id"],
            num_embeddings=items,
            embedding_dim=64,
            pooling="sum"
        )
    ]
)
```
3. **模型初始化**：
```python
from torchrec.models.dlrm import DLRM
model = DLRM(
    embedding_bag_collection=embedding_bag_collection,
    num_dense_features=num_dense,
    dense_layer_dims=[256, 128, 64],
    over_arch_layer_dims=[64, 1]
)
```
4. **训练设置**：
   - 优化器：`torchrec.optim.keyed.Lambda` 或 `torch.optim.Adagrad`；
   - 分布式：用 `torch.distributed.run` 启动，结合 `FullyShardedDataParallel` 管理显存。
5. **指标与校验**：离线计算 AUC/LogLoss；在线实验关注点击率、GMV；对曝光数据进行负采样策略评估。
6. **扩展**：加入多任务（CTR + CVR）、特征交叉（CrossNet）、延迟反馈校正（Delayed Feedback）。


### 9.6 什么时候选择其他结构？

- **图像相关任务**（检测、分割）：可在 9.1 的骨架上替换为 `torchvision.models.detection`（FasterRCNN、MaskRCNN）或 `torchvision.models.segmentation`（DeepLabV3）。
- **语音/音频生成**：基于 9.3 增加自回归解码器（`torchaudio.models.Tacotron2`），训练流程与分类类似，但损失改为 L1/stop token。
- **生成式/大型语言模型**：可使用 `TorchTune` 或 HuggingFace PEFT 在 9.2 的基础上扩展，加入 LoRA、量化。
- **时序预测/表格建模**：利用 `nn.Transformer`、`Temporal Fusion Transformers`；遵循同样步骤：定义数据窗口 → 构建模型 → 选择损失 → 分析误差。

> 实践提示：无论任务差异多大，核心循环始终是“数据准备 → 模型构建 → 损失优化 → 评估分析 → 迭代部署”。掌握这一套路后，换任务只需更换数据处理和模型细节。
